# Mosaic ArcticDEM strips captured the same day, reproject to Alaska Albers

In [ ]:
import subprocess
import glob
import os
from tqdm.auto import tqdm

In [ ]:
# Define DEM file names
arcticdem_path = '/Users/raineyaberle/Research/Hubbard/DEMs/ArcticDEM/'
arcticdem_folders = sorted(glob.glob(os.path.join(arcticdem_path, 'WV0*')))
arcticdem_fns = []
for folder in arcticdem_folders:
    arcticdem_fns += sorted(glob.glob(os.path.join(folder, '*dem_masked_clip.tif')))
print(f'Total number of ArcticDEM strips = {len(arcticdem_fns)}')

# Define target CRS for output DEMs
crs = 'EPSG:3338' # Alaska Albers

# Define output path
out_path = os.path.join(arcticdem_path, 'mosaics')

In [ ]:
# Check if out_path exists
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Made directory for output files:', out_path)

# Grab unique image dates from file names
im_dates = [os.path.basename(fn)[18:26] for fn in arcticdem_fns]
im_dates_unique = sorted(list(set(im_dates)))

# Iterate over unique image dates
for im_date in tqdm(im_dates_unique):
    # grab all files names with image date in file name
    dem_fns_date = [x for x in arcticdem_fns if im_date in os.path.basename(x)]
        
    # define output file name
    mosaic_fn = os.path.join(out_path, im_date + '_ArcticDEM_mosaic.tif')
    
    # check if file already exists in directory
    if os.path.exists(mosaic_fn):
        print('Mosaic already exists for date, skipping...')
        continue
        
    # construct gdal command
    cmd = 'gdalwarp '
    # add input file names
    for dem_fn in dem_fns_date:
        cmd += dem_fn + ' '
    # add target CRS
    cmd += f'-t_srs {crs} '
    # add output file name
    cmd += ' ' + mosaic_fn
    # print(cmd)
    
    # run command
    output = subprocess.run(cmd, shell=True, capture_output=True)
    print(output)
        
    print(' ')

In [ ]:
# Plot some example mosaics

import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import numpy as np

out_fns = sorted(glob.glob(os.path.join(out_path, '*.tif')))
for fn in out_fns[0:10]:
    im = rxr.open_rasterio(fn)
    im = xr.where(im==im.attrs['_FillValue'], np.nan, im)
    plt.imshow(im.data[0], cmap='terrain', clim=(0,1500),
               extent=(np.min(im.x.data), np.max(im.x.data), 
                       np.min(im.y.data), np.max(im.y.data)))
    plt.title(os.path.basename(fn))
    plt.colorbar(shrink=0.7, label='meters')
    plt.show()
im